In [9]:
import gc
import sys
from statistics import mean
import time
import torch
from tqdm import tqdm
from torchinfo import summary
import numpy as np
import itertools
from pathlib import Path
from scipy import special
import matplotlib.pyplot as plt
import datetime
import torch.linalg as LA
from torch.distributions.laplace import Laplace



import utils
import models
import learning_utils
from configurations import args_parser, arguments



In [10]:
%load_ext autoreload
%autoreload 2



args = arguments()
#boardio is for the the tensorboardx prensetaion and textio is for written documentation
boardio, textio, best_val_acc, path_best_model, last_model_path = utils.initializations(args)
textio.cprint(str(args) if args.__class__.__name__ == 'Namespace' else str(vars(args)))





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'eval': <built-in function eval>, 'data': 'cifar10', 'model': 'cnn3', 'num_users': 30, 'num_users_per_round': 5, 'local_epochs': 1, 'local_iterations': 100, 'global_epochs': 300, 'tau_min': 0.05, 'privacy_noise': 'laplace', 'epsilon_bar': 100, 'optimizer': 'Adam', 'lr': 0.01, 'momentum': 0.5, 'lr_scheduler': False, 'seed': 0, 'alpha': 100, 'beta': 2, 'gamma': 5, 'max_seconds': 300, 'method_choosing_users': 'all users', 'data_truncation': None, 'choosing_users_verbose': False, 'save_best_model': False, 'privacy': False, 'privacy_choosing_users': True, 'epsilon_sum_deascent_coeff': 0.04, 'delta_f': 0.012, 'device': device(type='cuda'), 'snr_verbose': True, 'max_iterations_alsa': 500, 'ALSA_simulation': False, 'ALSA_verbose': False, 'beta_max_reduction': 30, 'accel_ucb_coeff': 1, 'max_time_alsa': 600, 'pre_ALSA_rounds': 1, 'norm_std': 0.5, 'norm_mean': 0.5, 'train_batch_size': 20, 'test_batch_size': 1

In [11]:
# data
train_data, test_loader = utils.data(args)
#input in the CNNs is the number of channels and in linear models is the size of the flatten pictures
input, output, train_data, val_loader = utils.data_split(train_data, len(test_loader.dataset), args)

# model
if args.model == 'mlp':
    global_model = models.FC3Layer(input, output)
elif args.model == 'cnn2':
    global_model = models.CNN2Layer(input, output, args.data)
elif args.model == 'cnn3':
    if args.data == 'cifar10':
        global_model = models.CNN3LayerCifar()
    else:
        global_model = models.CNN3LayerMnist()
elif args.model == 'cnn5':
    if args.data == 'mnist' or args.data == 'fashion mnist':
        raise ValueError('CNN5 is not supported for MNIST type datasets')
    global_model = models.CNN5Layer(input, output)
elif args.model == 'linear':
    global_model = models.Linear(input, output)



textio.cprint(str(summary(global_model)).encode('utf-8', errors='ignore').decode('utf-8', errors='ignore'))
global_model = global_model.to(args.device)
print(f"global model's device: {next(global_model.parameters()).device}")

train_criterion = torch.nn.CrossEntropyLoss(reduction='mean')
test_criterion = torch.nn.CrossEntropyLoss(reduction='sum')

local_models = utils.federated_setup(global_model, train_data, args, i_i_d=True)
utils.update_data_equility_partititon(local_models, args)





100%|██████████| 170498071/170498071 [00:20<00:00, 8400051.27it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Layer (type:depth-idx)                   Param #
CNN3LayerCifar                           --
├─Conv2d: 1-1                            896
├─BatchNorm2d: 1-2                       64
├─Conv2d: 1-3                            18,496
├─BatchNorm2d: 1-4                       128
├─Conv2d: 1-5                            73,856
├─BatchNorm2d: 1-6                       256
├─MaxPool2d: 1-7                         --
├─Linear: 1-8                            65,664
├─BatchNorm1d: 1-9                       256
├─Linear: 1-10                           8,256
├─BatchNorm1d: 1-11                      128
├─Linear: 1-12                           2,080
├─BatchNorm1d: 1-13                      64
├─Linear: 1-14                           330
├─Dropout: 1-15                          --
├─Dropout: 1-16                          --
├─Dropout: 1-17                          --
Total params: 170,474
Trainable params: 170,474
Non-trainable params: 0
global model

In [12]:
# for i in range(100):
#     local_models[0].update_privacy_violation_and_reward()
#     print(f"iteration {i}, next_privacy_term is {local_models[0].next_privacy_term}")

In [13]:


choices_table = np.zeros((args.global_epochs, args.num_users))
num_of_obs_arr = np.zeros((1,args.num_users))
train_loss_list = []
val_acc_list = []
val_losses_list = []
l1_norm_avg_deltha_theta_list = []
global_epochs_time_list = []
privacy_violations_list = []
max_privacy_violation = 0


time_counter = 0
for global_epoch in tqdm(range(1, args.global_epochs+1)):
    """Part 1: Choosing Users"""
    for usr_idx in range(args.num_users):
        local_models[usr_idx].update_g(global_epoch)
        local_models[usr_idx].update_ucb(global_epoch)
    
    if args.choosing_users_verbose:
        textio.cprint(f"iteration: {global_epoch}")
    a=time.time()
    rounds_choise = utils.choose_users(local_models, args, global_epoch, textio, method=args.method_choosing_users)

    
    #choices_table[global_epoch-1, rounds_choise] = 1
    num_of_obs_arr[0,rounds_choise] += 1
    num_slow_users = 0
    num_fast_users = 0
    for usr_idx in sorted(rounds_choise):
        local_models[usr_idx].update_emp_avg()
        local_models[usr_idx].update_privacy_violation_and_reward()
        local_models[usr_idx].increase_num_of_obs()
        if local_models[usr_idx].privacy_violation > max_privacy_violation:
            max_privacy_violation = local_models[usr_idx].privacy_violation
        if args.choosing_users_verbose:
            textio.cprint(f"user {usr_idx}, emp_avg: {local_models[usr_idx].emp_avg}, h: {local_models[usr_idx].ucb_generalization}, ucb: {local_models[usr_idx].ucb}, num_of_obs(calculated after picking at this round): {local_models[usr_idx].num_of_obs}, privacy reward: {local_models[usr_idx].privacy_reward}, g: {local_models[usr_idx].g}, curr_delay = {local_models[usr_idx].last_access_time}")
        if usr_idx < args.num_users//2:
            num_fast_users += 1
        else:
            num_slow_users += 1
    if args.choosing_users_verbose:
        textio.cprint(f"num of fast users chosen: {num_fast_users}, num of slow users chosen: {num_slow_users}")
    
    max_delay = max([local_models[i].last_access_time for i in rounds_choise])
    if args.choosing_users_verbose:
        textio.cprint(f"max_delay = {max_delay:.2f} seconds")

    privacy_violations_list.append(max_privacy_violation)
    
    
    
    """Part 2: Training"""
    learning_utils.distribute_model(local_models, global_model)
    users_avg_loss_over_local_epochs = []

    for user_idx in rounds_choise:
        user_loss = []
        for local_epoch in range(args.local_epochs):
            user = local_models[user_idx]
            train_loss = learning_utils.train_one_epoch(user, train_criterion, args)
            if args.lr_scheduler:
                user.scheduler.step(train_loss)
            user_loss.append(train_loss)
        users_avg_loss_over_local_epochs.append(mean(user_loss))
    
    avg_loss_over_chosen_users_curr_global_epoch = mean(users_avg_loss_over_local_epochs)
    train_loss_list.append(avg_loss_over_chosen_users_curr_global_epoch)


    avg_deltha_theta = learning_utils.Fed_avg_models(local_models, global_model, rounds_choise, textio
                                                     ,args, snr_verbose = args.snr_verbose)
    # l1_norm_avg_deltha_theta = sum(LA.vector_norm(param.flatten(),1) for param in avg_deltha_theta.values())
    # l1_norm_avg_deltha_theta_list.append(l1_norm_avg_deltha_theta)
    # print(f"l1_norm_avg_deltha_theta is {l1_norm_avg_deltha_theta}")
    

    val_acc, val_loss = learning_utils.test(test_loader, global_model, test_criterion, args)
    val_acc_list.append(val_acc) ; val_losses_list.append(val_loss)
    

    # boardio.add_scalars("Losses over time in seconds", {"train_loss":avg_loss_over_chosen_users_curr_global_epoch,
    #                                     "val_loss": val_loss}, time.time()-start_time)
    # boardio.add_scalar('Val Accuracy', val_acc, time.time()-start_time)


    time_counter += max_delay
    textio.cprint((f"global epoch {global_epoch} has been done artifficialy in {max_delay:.2f} secs, the total time by now is {time_counter:.2f} \n with avg train loss {avg_loss_over_chosen_users_curr_global_epoch:.3f}, val loss {val_loss:.3f}, avg val acc {val_acc:.2f}%"))
    global_epochs_time_list.append(time_counter)
    gc.collect()


    if val_acc > best_val_acc and args.save_best_model:
        best_val_acc = val_acc
        torch.save({"model's state dict":global_model.state_dict(),
                    "train_loss_list": train_loss_list,
                    "val_acc_list": val_acc_list,
                    "val_losses_list": val_losses_list,
                    "global_epochs_time_list": global_epochs_time_list,
                    "num_of_users": args.num_users,
                    "num_of_users_per_round": args.num_users_per_round,
                    "privacy_violations_list": privacy_violations_list}
                    , path_best_model)
    
    
    torch.save({"model's state dict":global_model.state_dict(),
                "train_loss_list": train_loss_list,
                "val_acc_list": val_acc_list,
                "val_losses_list": val_losses_list,
                "global_epochs_time_list": global_epochs_time_list,
                "num_of_obs_arr": num_of_obs_arr.reshape(-1),
                "global_epoch": global_epoch,
                "num_of_users": args.num_users,
                "num_of_users_per_round": args.num_users_per_round,
                "privacy_violations_list": privacy_violations_list}
                , last_model_path)
    
    users_idxs = tuple([str(x) for x in range(1,args.num_users+1)])
    plt.bar(users_idxs, num_of_obs_arr.reshape(-1))
    plt.title("Number of times each user was chosen")
    plt.ylabel("number of times")
    #hide x labels 
    plt.xticks([])
    fig = plt.gcf()
    fig.set_size_inches(18.5, 10.5)
    #boardio.add_figure("Number of times each user was chosen", fig, global_epoch)
    plt.savefig(last_model_path.parent / "Number of times each user was chosen.png")

    if time_counter > args.max_seconds:
        break




  0%|          | 0/300 [00:00<?, ?it/s]

after cleansing the weights updates who were almost zeros (meaning with absolute value of less than 10**-7, 0.00% of the weights upadtes) we get:
user No.2 has been picked 1, his deltas theta's mean
 is -0.0006463671452365816,
the empirical var of deltas thetas is 0.003982466645538807
the minimum of the deltas theta is -0.25982600450515747 and the maximum is 0.34222981333732605
the added laplace noise var is 0.003060399989333737
global epoch 1 has been done artifficialy in 0.92 secs, the total time by now is 0.92 
 with avg train loss 2.076, val loss 2.365, avg val acc 10.00%


  0%|          | 1/300 [00:23<1:57:46, 23.64s/it]

after cleansing the weights updates who were almost zeros (meaning with absolute value of less than 10**-7, 0.00% of the weights upadtes) we get:
user No.23 has been picked 2, his deltas theta's mean
 is -0.00027860471163876355,
the empirical var of deltas thetas is 0.001486930064857006
the minimum of the deltas theta is -0.2237623929977417 and the maximum is 0.294674813747406
the added laplace noise var is 0.0031852972822368235
global epoch 2 has been done artifficialy in 0.94 secs, the total time by now is 1.86 
 with avg train loss 1.958, val loss 2.261, avg val acc 12.63%


  1%|          | 2/300 [00:43<1:47:02, 21.55s/it]

...

 84%|████████▍ | 252/300 [1:37:35<19:04, 23.84s/it]

after cleansing the weights updates who were almost zeros (meaning with absolute value of less than 10**-7, 0.00% of the weights upadtes) we get:
user No.27 has been picked 253, his deltas theta's mean
 is -0.0008115317905321717,
the empirical var of deltas thetas is 0.002196544548496604
the minimum of the deltas theta is -0.31150564551353455 and the maximum is 0.2914765477180481
the added laplace noise var is 73.02415989853215
global epoch 253 has been done artifficialy in 0.91 secs, the total time by now is 237.44 
 with avg train loss 0.112, val loss 1.139, avg val acc 73.68%


 84%|████████▍ | 253/300 [1:38:00<18:52, 24.10s/it]

after cleansing the weights updates who were almost zeros (meaning with absolute value of less than 10**-7, 0.00% of the weights upadtes) we get:
user No.11 has been picked 254, his deltas theta's mean
 is -0.0003566762898117304,
the empirical var of deltas thetas is 0.005501358304172754
the minimum of the deltas theta is -0.36987927556037903 and the maximum is 0.4506489634513855
the added laplace noise var is 76.00433239873992
global epoch 254 has been done artifficialy in 0.91 secs, the total time by now is 238.36 
 with avg train loss 0.120, val loss 1.142, avg val acc 73.43%


 85%|████████▍ | 254/300 [1:38:24<18:35, 24.25s/it]